In [2]:
#%pip install jellyfish
import jellyfish as jf

In [2]:
jf.levenshtein_distance('Michael','Micheal')

2

In [3]:
min(jf.levenshtein_distance('el','ael'), jf.levenshtein_distance('ael','al'), jf.levenshtein_distance('el','al'))

1

In [4]:
jf.jaro_similarity('Michael','Micheal')

0.9523809523809524

In [5]:
jf.jaro_winkler_similarity('Michael','Micheal')

0.9714285714285714

In [6]:
jf.jaro_winkler_similarity('michael','MICHAEL')

0.0

In [7]:
jf.jaro_winkler_similarity('michael'.lower(),'MICHAEL'.lower())

1.0

In [16]:
jf.metaphone('michael')

'MXL'

In [17]:
jf.metaphone('micheal')

'MXL'

In [4]:
import pandas as pd

In [12]:
mylist = ['Michael','Micheal','Michel','Mike','Mick']

In [13]:
combs = []
import itertools
for a, b in itertools.combinations(mylist, 2):
    combs.append([a,b,jf.jaro_similarity(a,b),jf.jaro_winkler_similarity(a, b), jf.levenshtein_distance(a,b), jf.match_rating_comparison(a,b),(jf.soundex(a)==jf.soundex(b))])
pd.DataFrame(combs, columns=['Name1','Name2','Jaro','JaroW','Levenshtein','MRA','Soundex'])

,Name1,Name2,Jaro,JaroW,Levenshtein,MRA,Soundex
0,Michael,Micheal,0.952381,0.971429,2,True,True
1,Michael,Michel,0.952381,0.971429,1,True,True
2,Michael,Mike,0.726190,0.780952,4,False,False
3,Michael,Mick,0.726190,0.808333,4,True,False
4,Micheal,Michel,0.952381,0.971429,1,True,True
5,Micheal,Mike,0.726190,0.780952,4,False,False
6,Micheal,Mick,0.726190,0.808333,4,True,False
7,Michel,Mike,0.750000,0.800000,3,False,False
8,Michel,Mick,0.750000,0.825000,3,True,False
9,Mike,Mick,0.833333,0.866667,2,True,True


In [5]:
df_mp_un = pd.read_csv('mps_unmatched.csv')
df_mp_un

,Constituency,Firstname_x,Lastname_x,Firstname_y,Lastname_y
0,Burton,Kate,Griffiths,Kate,Kniveton
1,Central Suffolk and North Ipswich,Dan,Poulter,Daniel,Poulter
2,Newton Abbot,Anne,Marie Morris,Anne,Morris
3,North Antrim,Ian,Paisley,Ian,Paisley Jnr
4,Slough,Tanmanjeet,Dhesi,Tan,Dhesi
5,South Down,Chris,Hazzard,Christopher,Hazzard
6,South West Norfolk,Liz,Truss,Elizabeth,Truss
7,Wealden,Nus,Ghani,Nusrat,Ghani
8,West Dunbartonshire,Martin,Docherty-Hughes,Martin,Docherty


In [12]:
df_mp_un['Firstname_jaro'] = df_mp_un.apply(lambda x: jf.jaro_winkler_similarity(x.Firstname_x, x.Firstname_y), axis=1)
df_mp_un['Lastname_jaro'] = df_mp_un.apply(lambda x: jf.jaro_winkler_similarity(x.Lastname_x, x.Lastname_y), axis=1)

In [13]:
df_mp_un[(df_mp_un['Firstname_jaro'] > 0.8) & (df_mp_un['Lastname_jaro'] > 0.8)]

,Constituency,Firstname_x,Lastname_x,Firstname_y,Lastname_y,Firstname_jaro,Lastname_jaro
1,Central Suffolk and North Ipswich,Dan,Poulter,Daniel,Poulter,0.883333,1.000000
3,North Antrim,Ian,Paisley,Ian,Paisley Jnr,1.000000,0.927273
4,Slough,Tanmanjeet,Dhesi,Tan,Dhesi,0.836667,1.000000
5,South Down,Chris,Hazzard,Christopher,Hazzard,0.890909,1.000000
7,Wealden,Nus,Ghani,Nusrat,Ghani,0.883333,1.000000
8,West Dunbartonshire,Martin,Docherty-Hughes,Martin,Docherty,1.000000,0.906667


In [14]:
import pandas as pd

df = pd.read_csv('mps_wiki_clean.csv')
df_mp = pd.read_csv('mps_they_clean.csv')

cross = df.merge(df_mp, how='cross',suffixes=('_w', '_t'))

In [15]:
cross.head(n=5)

,Constituency_w,Firstname_w,Lastname_w,Constituency_t,Firstname_t,Lastname_t
0,Aberavon,Stephen,Kinnock,Hackney North and Stoke Newington,Diane,Abbott
1,Aberavon,Stephen,Kinnock,Oldham East and Saddleworth,Debbie,Abrahams
2,Aberavon,Stephen,Kinnock,Selby and Ainsty,Nigel,Adams
3,Aberavon,Stephen,Kinnock,Hitchin and Harpenden,Bim,Afolami
4,Aberavon,Stephen,Kinnock,Windsor,Adam,Afriyie


In [16]:
cross['Firstname_jaro'] = cross.apply(lambda x: True if jf.jaro_winkler_similarity(x.Firstname_w, x.Firstname_t)>0.8 else False, axis=1)
cross['Lastname_jaro'] = cross.apply(lambda x: True if jf.jaro_winkler_similarity(x.Lastname_w, x.Lastname_t)>0.8 else False, axis=1)

KeyboardInterrupt: 

In [ ]:
cross.head(n=5)

In [ ]:
tp = cross[(cross['Firstname_jaro'] & cross['Lastname_jaro']) & (cross['Constituency_w']==cross['Constituency_t'])]
len(tp)

In [ ]:
fp = cross[(cross['Firstname_jaro'] & cross['Lastname_jaro']) & (cross['Constituency_w']!=cross['Constituency_t'])]
len(fp)

In [ ]:
fntn = cross[(~cross['Firstname_jaro'] | ~cross['Lastname_jaro']) & (cross['Constituency_w']==cross['Constituency_t'])]
len(fntn)

In [ ]:
fntn

In [1]:
df['Firstname'].value_counts().mean()

NameError: name 'df' is not defined

In [38]:
df['Lastname'].value_counts().mean()

1.1545293072824157

In [39]:
df['Constituency'].value_counts().mean()

1.0

In [6]:
df_mp_un['Firstname_x_meta'] = df_mp_un.apply(lambda x: jf.metaphone(x.Firstname_x), axis=1)

In [7]:
df_mp_un

,Constituency,Firstname_x,Lastname_x,Firstname_y,Lastname_y,Firstname_x_meta
0,Burton,Kate,Griffiths,Kate,Kniveton,KT
1,Central Suffolk and North Ipswich,Dan,Poulter,Daniel,Poulter,TN
2,Newton Abbot,Anne,Marie Morris,Anne,Morris,AN
3,North Antrim,Ian,Paisley,Ian,Paisley Jnr,IN
4,Slough,Tanmanjeet,Dhesi,Tan,Dhesi,TNMNJT
5,South Down,Chris,Hazzard,Christopher,Hazzard,XRS
6,South West Norfolk,Liz,Truss,Elizabeth,Truss,LS
7,Wealden,Nus,Ghani,Nusrat,Ghani,NS
8,West Dunbartonshire,Martin,Docherty-Hughes,Martin,Docherty,MRTN


In [8]:
df_mp_un['Firstname_y_meta'] = df_mp_un.apply(lambda x: jf.metaphone(x.Firstname_y), axis=1)

In [9]:
df_mp_un

,Constituency,Firstname_x,Lastname_x,Firstname_y,Lastname_y,Firstname_x_meta,Firstname_y_meta
0,Burton,Kate,Griffiths,Kate,Kniveton,KT,KT
1,Central Suffolk and North Ipswich,Dan,Poulter,Daniel,Poulter,TN,TNL
2,Newton Abbot,Anne,Marie Morris,Anne,Morris,AN,AN
3,North Antrim,Ian,Paisley,Ian,Paisley Jnr,IN,IN
4,Slough,Tanmanjeet,Dhesi,Tan,Dhesi,TNMNJT,TN
5,South Down,Chris,Hazzard,Christopher,Hazzard,XRS,XRSTFR
6,South West Norfolk,Liz,Truss,Elizabeth,Truss,LS,ELSB0
7,Wealden,Nus,Ghani,Nusrat,Ghani,NS,NSRT
8,West Dunbartonshire,Martin,Docherty-Hughes,Martin,Docherty,MRTN,MRTN


In [10]:
df_mp_un['Lastname_x_meta'] = df_mp_un.apply(lambda x: jf.metaphone(x.Lastname_x), axis=1)
df_mp_un['Lastname_y_meta'] = df_mp_un.apply(lambda x: jf.metaphone(x.Lastname_y), axis=1)

In [11]:
df_mp_un

,Constituency,Firstname_x,Lastname_x,Firstname_y,Lastname_y,Firstname_x_meta,Firstname_y_meta,Lastname_x_meta,Lastname_y_meta
0,Burton,Kate,Griffiths,Kate,Kniveton,KT,KT,KRF0S,NFTN
1,Central Suffolk and North Ipswich,Dan,Poulter,Daniel,Poulter,TN,TNL,PLTR,PLTR
2,Newton Abbot,Anne,Marie Morris,Anne,Morris,AN,AN,MR MRS,MRS
3,North Antrim,Ian,Paisley,Ian,Paisley Jnr,IN,IN,PSL,PSL JNR
4,Slough,Tanmanjeet,Dhesi,Tan,Dhesi,TNMNJT,TN,THS,THS
5,South Down,Chris,Hazzard,Christopher,Hazzard,XRS,XRSTFR,HSRT,HSRT
6,South West Norfolk,Liz,Truss,Elizabeth,Truss,LS,ELSB0,TRS,TRS
7,Wealden,Nus,Ghani,Nusrat,Ghani,NS,NSRT,KHN,KHN
8,West Dunbartonshire,Martin,Docherty-Hughes,Martin,Docherty,MRTN,MRTN,TXRTHKHS,TXRT


In [15]:
combs = []
import itertools
for a, b in itertools.combinations(mylist, 2):
    combs.append([a,b,jf.jaro_similarity(a,b),jf.jaro_winkler_similarity(a, b), jf.levenshtein_distance(a,b), jf.match_rating_comparison(a,b),(jf.soundex(a)==jf.soundex(b)), jf.metaphone(a), jf.metaphone(b)])
pd.DataFrame(combs, columns=['Name1','Name2','Jaro','JaroW','Levenshtein','MRA','Soundex','Name1Meta','Name2Meta'])

,Name1,Name2,Jaro,JaroW,Levenshtein,MRA,Soundex,Name1Meta,Name2Meta
0,Michael,Micheal,0.952381,0.971429,2,True,True,MXL,MXL
1,Michael,Michel,0.952381,0.971429,1,True,True,MXL,MXL
2,Michael,Mike,0.726190,0.780952,4,False,False,MXL,MK
3,Michael,Mick,0.726190,0.808333,4,True,False,MXL,MK
4,Micheal,Michel,0.952381,0.971429,1,True,True,MXL,MXL
5,Micheal,Mike,0.726190,0.780952,4,False,False,MXL,MK
6,Micheal,Mick,0.726190,0.808333,4,True,False,MXL,MK
7,Michel,Mike,0.750000,0.800000,3,False,False,MXL,MK
8,Michel,Mick,0.750000,0.825000,3,True,False,MXL,MK
9,Mike,Mick,0.833333,0.866667,2,True,True,MK,MK
